<a href="https://colab.research.google.com/github/Praveen-Progs-Hub-2018/Text-Processing-with-NLTK/blob/master/BERT_MorningStar_NER.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Import Libraries**

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
eng_train_file = "eng_train.txt"
eng_train_tag_file = "eng_train_tag.txt"

**Loading Input files**

In [10]:
sentences = []
file1=open(eng_train_file,"r")
file2=open(eng_train_tag_file,"r")
for index in range(1000):
    line1=next(file1).split()
    line2 =next(file2).split()
    for i in range(len(line1)):
      sentences.append([index+1, line1[i], line2[i]])
data = pd.DataFrame(sentences, columns=['sentence_id','words','labels'])
data.head(4)

,sentence_id,words,labels
0,1,EU,B-ORG
1,1,rejects,O
2,1,German,B-MISC
3,1,call,O


In [11]:
data["labels"] = data["labels"].str.upper()
X= data[["sentence_id","words"]]
Y =data["labels"]
x_train, x_test, y_train, y_test = train_test_split(X,Y, test_size =0.2)

**Building up train data and test data**

In [12]:
train_data = pd.DataFrame({"sentence_id":x_train["sentence_id"],"words":x_train["words"],"labels":y_train})
test_data = pd.DataFrame({"sentence_id":x_test["sentence_id"],"words":x_test["words"],"labels":y_test})

In [13]:
!pip install simpletransformers

In [14]:
from simpletransformers.ner import NERModel,NERArgs
from sklearn.metrics import accuracy_score

In [15]:
label = data["labels"].unique().tolist()
label

['B-ORG', 'O', 'B-MISC', 'B-PER', 'I-PER', 'B-LOC', 'I-ORG', 'I-MISC', 'I-LOC']

**Prepare Model Args**

In [16]:
args = NERArgs()
args.num_train_epochs = 1
args.learning_rate = 1e-4
args.overwrite_output_dir =True
args.train_batch_size = 32
args.eval_batch_size = 32
model = NERModel('bert', 'bert-base-cased',labels=label,args =args, use_cuda=False)

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForTokenClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-cas

In [17]:
from sklearn.metrics import accuracy_score


**Model Training**

In [18]:
model.train_model(train_data,eval_data = test_data,acc=accuracy_score)

  0%|          | 0/2 [00:00<?, ?it/s]

Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Running Epoch 0 of 1:   0%|          | 0/31 [00:00<?, ?it/s]

(31, 0.699939678753576)

**Model Evaluation**

In [19]:
result, model_outputs, preds_list = model.eval_model(test_data)

  0%|          | 0/2 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/25 [00:00<?, ?it/s]

In [21]:
result

{'eval_loss': 0.388534922003746,
 'precision': 0.6777163904235728,
 'recall': 0.6548042704626335,
 'f1_score': 0.6660633484162896}

In [22]:
prediction, model_output = model.predict(["What is the new name of Bangalore"])
prediction

  0%|          | 0/1 [00:00<?, ?it/s]

Running Prediction:   0%|          | 0/1 [00:00<?, ?it/s]

[[{'What': 'O'},
  {'is': 'O'},
  {'the': 'O'},
  {'new': 'O'},
  {'name': 'O'},
  {'of': 'O'},
  {'Bangalore': 'B-LOC'}]]